In [ ]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import check_random_state
from sklearn.metrics import mean_squared_error

!pip install pmdarima
from pmdarima import auto_arima

# This will help to eliminate some irritating warnings
pd.options.mode.copy_on_write = True

In [ ]:

class GBVARMAX:
    """
    Gradient Boosting Vector Autoregressive Moving Average with eXogeneous Variables

    This class implements a Gradient Boosting algorithm for regression with 
    lagged dependent variables. It supports 
    multiple output variables and includes features such as:

    - **Lagged dependent variables:** Incorporates past values of the target 
      variables as features.
    - **Lagged prediction errors:** Uses past prediction errors as features.
    - **Validation split:** Enables automatic splitting of data for model 
      evaluation during training.

    Parameters
    ----------
    n_estimators : int, default=100
        The number of boosting stages to perform.

    learning_rate : float, default=0.01
        Learning rate shrinks the contribution of each tree by `learning_rate`.

    max_depth : int, default=5
        Maximum depth of the individual regression trees.

    n_lags : int, default=4
        The number of lags of the target variable to include as features.

    random_state : int or None, default=None
        Controls the randomness of the estimator.

    validation_split : float, default=0.2
        The proportion of the training set to use for validation.

    """

    def __init__(self, n_estimators=200, learning_rate=0.01, max_depth=4, 
                 n_lags=4, random_state=None, validation_split=0.2):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.n_lags = n_lags
        self.random_state = random_state
        self.validation_split = validation_split
        self.trees_ = []
        self.val_scores_ = np.array([])   # List to store validation scores for each output
        self.train_scores_ = np.array([]) # List to store training scores for each output

    def _create_lagged_features(self, y, y_pred=None, x0=None, x1=None):
        """
        Creates lagged features of the target variable and prediction errors using pandas.DataFrame.shift().

        Args:
          y: pandas.DataFrame of shape (n_samples, n_outputs)
              Target values.
          y_pred: array-like of shape (n_samples, n_outputs)
              Predicted values. 

        Returns:
          lagged_features: array-like 
              Array containing lagged values of y and prediction errors.
        """
        lagged_df = y.copy()
        for i in range(1, self.n_lags + 1):
            for col in lagged_df.columns:
                lagged_df[f'{col}_lag_{i}'] = lagged_df[col].shift(i)
                lagged_df = lagged_df.copy()
                
        # Fill NaN values in lagged features with the mean of the corresponding column 
        for col in lagged_df.columns:
            lagged_df[col] = lagged_df[col].fillna(lagged_df[col].mean()) 
            
        lagged_features = lagged_df.values 

        if y_pred is None:
            y_pred = np.tile(self.y_mean_, (y.shape[0], 1))
        lagged_pred_df = pd.DataFrame(y_pred)
        for i in range(1, self.n_lags + 1):
            for col in lagged_pred_df.columns:
                lagged_pred_df[f'{col}_pred_lag_{i}'] = lagged_pred_df[col].shift(i)
                lagged_pred_df = lagged_pred_df.copy()
        lagged_errors = lagged_df.values - lagged_pred_df.values 
            
        # If we don't know what the error is, use 0
        lagged_errors = np.nan_to_num(lagged_errors, nan=0)
            
        lagged_features = np.concatenate((lagged_features, lagged_errors), axis=1)
        return pd.DataFrame(lagged_features)

    def fit(self, y, x=None, x_d=None):
        """
        Fit the Gradient Boosting VARMAX model.

        Parameters
        ----------
        y : pandas.DataFrame of shape (n_samples, n_outputs)
            Training target values, where n_outputs is the number of output variables. 

        x : pandas.Dataframe of shape (n_samples, n_exog)
            Exogenous predictors

        x_d : dict of pandas.Dataframe of shape (n_samples, n_exog_i)
            Exogeneous predictors for specific y_i

        Returns
        -------
        self : object
            Fitted estimator.
        """
        if not isinstance(y, pd.DataFrame):
            raise ValueError("Input 'y' must be a pandas DataFrame.")

        n_samples_train = y.shape[0]
        n_outputs = y.shape[1] 

        # Generate random indices for train/validation split
        indices = np.arange(n_samples_train)
        rng = check_random_state(self.random_state)
        rng.shuffle(indices)
        train_size = int((1 - self.validation_split) * n_samples_train)
        train_indices = indices[:train_size]
        val_indices = indices[train_size:]

        # Calculate mean on training data only
        self.y_mean_ = np.mean(y.iloc[train_indices], axis=0) 
        y_pred = y.copy() 
        y_pred.iloc[:] = self.y_mean_ 


        self.val_scores_ = np.zeros((self.n_estimators, n_outputs))
        self.train_scores_ = np.zeros((self.n_estimators, n_outputs))
        for i in range(self.n_estimators):

            # Create lagged features for the training data 
            lagged_features_all = self._create_lagged_features(y, y_pred)
            lagged_features_train = lagged_features_all.iloc[train_indices].copy()

            # Fit a decision tree for each output variable
            self.trees_.append([])
#            print(">>>")
#            print(lagged_features_all.head(2))
#            print(lagged_features_all.tail(2))
            
            for j in range(n_outputs):
                tree = DecisionTreeRegressor(max_depth=self.max_depth)
                tree.fit(lagged_features_train, y.iloc[train_indices, j]) 
                self.trees_[-1].append(tree)

            # Update predictions for all data
            lagged_features_all = self._create_lagged_features(y_pred)
            for j in range(n_outputs):
                y_pred.iloc[:, j] -= self.learning_rate * self.trees_[-1][j].predict(lagged_features_all) 

            # Calculate validation error 
            y_pred_val = y_pred.iloc[val_indices]
            y_val = y.iloc[val_indices]
            for j in range(n_outputs):
                self.val_scores_[i, j] = mean_squared_error(y_val.iloc[:, j], y_pred_val.iloc[:,j])

            # Calculate training error
            y_pred_train = y_pred.iloc[train_indices]
            y_train = y.iloc[train_indices]
            for j in range(n_outputs):
                self.train_scores_[i, j] = mean_squared_error(y_train.iloc[:, j], y_pred_train.iloc[:,j])



            print("i = ", i, " of ", self.n_estimators)

        return self

    def forecast(self, y):
        """
        Forecast future values for y.

        Parameters
        ----------
        y : pandas.DataFrame of shape (n_samples, n_outputs)
            Historical target values for forecasting. 

        Returns
        -------
        y_forecast : array-like of shape (n_samples, n_outputs)
            Forecasted values for y.
        """
        n_outputs = len(self.trees_[0])
        y_forecast = np.full_like(y, self.y_mean_) 

        if not isinstance(y, pd.DataFrame):
            raise ValueError("Input 'y' must be a pandas DataFrame.")

        for i in range(self.n_lags, len(y)):
            lagged_features = self._create_lagged_features(y[:i], y_forecast[:i]) 
            for j in range(n_outputs):
                y_forecast[i, j] = self.y_mean_[j] + sum(tree.predict(lagged_features) for tree in self.trees_[:, j])

        return y_forecast

In [ ]:
tb = pd.read_csv("/kaggle/input/sharadar-etfs/sharadar/SFP.csv/SHARADAR_SFP_2_fb4f5d2244276f3cfeca03f46b122d99.csv")

    

In [ ]:
# This code isn't pretty, but it is thousands of times faster than the 
# easy-to-read alternative that calculates log return by ticker
tb = tb.sort_values(by=["ticker", "date"])
tb["revenue"] = tb["close"] * tb["volume"]
tb["return"] = tb["closeadj"] / tb["closeadj"].shift()
tb["tickeryesterday"] = tb["ticker"].shift()
tb["logreturn"] = np.log(tb["closeadj"] / tb["closeadj"].shift())
tb.loc[tb["ticker"] != tb["tickeryesterday"], "return"] = np.nan
tb.drop("tickeryesterday", axis=1, inplace=True)

In [ ]:
tb.head()

In [ ]:
def tickers_by_revenue(ltb):
    stickers = ltb.groupby("ticker")["revenue"].sum().sort_values(ascending=False).index.to_list()
    return stickers



In [ ]:
n_base_etfs = 200
n_model_etfs = 16

window_size = 1250
skip_size = 20

first_train_data_date = "2002-01-01"
first_val_data_date = "2019-01-01"
first_test_data_date = "2023-01-01"
first_eval_data_date = "2024-01-01"




In [ ]:
def gen_ts_vectors(stb, tickers):
    vs = []
    for t in tickers:
        v = stb.loc[stb["ticker"] == t, ["date", "logreturn"]]
        v = v.set_index("date", drop=True)
        vs.append(v)
    
    rv = pd.concat(vs, axis=1)
    rv.columns = tickers
    return rv


In [ ]:
tb_train = tb[(tb["date"] >= first_train_data_date) & (tb["date"] < first_val_data_date)]

In [ ]:
train_tickers = tickers_by_revenue(tb_train)[0:20]

train_endog = gen_ts_vectors(tb_train, train_tickers)


In [ ]:
train_endog.head()

In [ ]:
train_endog[np.isnan(train_endog)] = 0

In [ ]:
gbv = GBVARMAX()
gbv.fit(train_endog)

In [ ]:
xx = 0
pd.DataFrame(gbv.val_scores_).iloc[:,xx].plot(title="Error For " + train_endog.columns[xx] + " Log Returns", xlabel="# of Trees", ylabel="MSE")
